In [ ]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive/My Drive/Homework1

MessageError: ignored

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import numpy as np
import tensorflow as tf
from tensorflow import keras as tfk
from tensorflow.keras import layers as tfkl
import matplotlib.pyplot as plt
from random import randint

In [ ]:
seed = 25
import os
import random

random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)

In [ ]:
def load_data(folder="public_data.npz", resolution=96, head_only=False):
    images = []

    loaded = np.load(folder, allow_pickle=True)

    # Iterate through files in the specified folder
    for i, img in enumerate(loaded['data']):
        # Normalize image pixel values to a float range [0, 1]
        #img = (img / 255).astype(np.float32)

        # Convert image from BGR to RGB
        #img = img[...,::-1]

        # Make the image dataset squared
        dim = min(img.shape[:-1])
        img = img[(img.shape[0]-dim)//2:(img.shape[0]+dim)//2, (img.shape[1]-dim)//2:(img.shape[1]+dim)//2, :]

        # Resize the image to 224x224 pixels
        #img = tfkl.Resizing(224, 224)(img)
        img = tfkl.Resizing(resolution, resolution)(img)

        if img is not None:
            images.append(img)

        if (head_only and i == 9):
           break

    labels = loaded['labels']
    loaded.close()

    if (head_only):
       labels = labels[:10]

    y = LabelEncoder().fit_transform(labels)
    y = tfk.utils.to_categorical(y, 2)

    return np.array(images), y




def display_random_images(X, y, num_img=10):
  # Create subplots for displaying items
  fig, axes = plt.subplots(2, num_img//2, figsize=(20, 9))
  for i in range(num_img):
      image = randint(0, X.shape[0] - 1)

      ax = axes[i%2, i%num_img//2]
      ax.imshow(np.clip(X[image], 0, 255))  # Display clipped item images
      ax.text(0.5, -0.1, str(image) + ' ' + str(y[image]), size=12, ha="center", transform=ax.transAxes)
      ax.axis('off')
  plt.tight_layout()
  plt.show()




def delete_outliers(X, y):
  shrek = 137
  trololo = 5143

  new_X = []
  new_y = []

  num_outliers = 0

  for i, sample in enumerate(X):
    if (not (np.array_equal(sample, X[shrek]) or np.array_equal(sample, X[trololo]))):
      new_X.append(sample)
      new_y.append(y[i])
    else:
      num_outliers += 1

  return np.array(new_X), np.array(new_y), num_outliers

In [ ]:
X, y = load_data('public_data.npz')
X, y, num_outliers = delete_outliers(X, y)

In [ ]:
# Split data into train_val and test sets
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=500, stratify=np.argmax(y,axis=1))

# Further split train_val into train and validation sets
X_train, X_val, y_train_0, y_val = train_test_split(X_train_val, y_train_val, test_size=500, stratify=np.argmax(y_train_val,axis=1))

print(X_train.shape, y_train_0.shape)
print(X_val.shape, y_val.shape)
print(X_test.shape, y_test.shape)

(4004, 96, 96, 3) (4004, 2)
(500, 96, 96, 3) (500, 2)
(500, 96, 96, 3) (500, 2)


In [ ]:
augment1 = tf.keras.Sequential([
    tfkl.RandomFlip(),
    tfkl.RandomTranslation(height_factor = (-0.2,0.2), width_factor = (-0.2,0.2), fill_mode = 'reflect'),
    tfkl.RandomZoom(0.3, fill_mode = 'reflect'),
    tfkl.RandomBrightness(0.1, value_range=(0,255)),
])

augment2 = tf.keras.Sequential([
    tfkl.RandomFlip(),
    tfkl.RandomZoom(0.4, fill_mode = 'reflect'),
    tfkl.RandomBrightness(0.1, value_range=(0,255)),
    tfkl.RandomRotation((-1,1), fill_mode = 'reflect'),
])

new_X_train_1 = augment2(X_train[np.where((y_train_0[:, 0] == 0) & (y_train_0[:, 1] == 1))])
augmented_X_train_2 = augment2(X_train)
augmented_X_train_1 = augment1(augmented_X_train_2)

X_train = np.append(X_train ,augmented_X_train_2, axis = 0)
X_train = np.append(X_train ,augmented_X_train_1, axis = 0)
X_train = np.append(X_train ,new_X_train_1, axis = 0)

y_train = np.append(y_train_0, y_train_0, axis = 0)
y_train = np.append(y_train, y_train_0, axis = 0)
for k in range(new_X_train_1.shape[0]):
    y_train = np.append(y_train, [[0,1]], axis = 0)

In [ ]:
# Define key model parameters
input_shape = X_train.shape[1:]  # Input shape for the model
output_shape = y_train.shape[1]  # Output shape for the model
batch_size = 16                # Batch size for training, always a power of 2!!
epochs = 400

In [ ]:
mobile = tfk.applications.Xception(
    input_shape=(96, 96, 3),
    include_top=False,
    weights="imagenet",
    pooling='avg',
)
mobile.trainable = False

83683744/83683744 [==============================] - 0s 0us/step


In [ ]:
inputs = tfk.Input(shape=input_shape)
# Connect MobileNetV2 to the input
x = mobile(inputs)
x = tfkl.Dropout(rate = 1/5)(x)
x = tfkl.Dense(64)(x)
#x = tfkl.BatchNormalization()(x)
x = tfkl.Activation('relu')(x)
x = tfkl.Dropout(rate = 1/7)(x)
x = tfkl.Dense(16)(x)
#x = tfkl.BatchNormalization()(x)
x = tfkl.Activation('relu')(x)
x = tfkl.Dropout(rate = 1/7)(x)
# Add a Dense layer with 2 units and softmax activation as the classifier
outputs = tfkl.Dense(2, activation='softmax')(x)

# Create a Model connecting input and output
model_1 = tfk.Model(inputs=inputs, outputs=outputs, name='model')

# Compile the model with Categorical Cross-Entropy loss and Adam optimizer
model_1.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.AdamW(learning_rate=1e-3, weight_decay=3e-5), metrics=['accuracy'])

# Display model summary
model_1.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 96, 96, 3)]       0         
                                                                 
 xception (Functional)       (None, 2048)              20861480  
                                                                 
 dropout (Dropout)           (None, 2048)              0         
                                                                 
 dense (Dense)               (None, 64)                131136    
                                                                 
 activation (Activation)     (None, 64)                0         
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 16)                1040  

In [ ]:
history = model_1.fit(
    x = tfk.applications.xception.preprocess_input(X_train), # We need to apply the preprocessing thought for the MobileNetV2 network
    y = y_train,
    batch_size = 16,
    epochs = 400,
    validation_data = (tfk.applications.xception.preprocess_input(X_val), y_val), # We need to apply the preprocessing thought for the MobileNetV2 network
    callbacks = [
        tfk.callbacks.EarlyStopping(monitor='val_accuracy', mode='max', patience=30, restore_best_weights=True),
        tfk.callbacks.ReduceLROnPlateau(monitor = 'val_accuracy', factor = 0.9, patience = 8, min_lr = 2e-6)
    ]
).history

Epoch 1/400
846/846 [==============================] - 34s 24ms/step - loss: 0.5963 - accuracy: 0.6818 - val_loss: 0.4892 - val_accuracy: 0.7840 - lr: 0.0010
Epoch 2/400
846/846 [==============================] - 17s 21ms/step - loss: 0.5448 - accuracy: 0.7252 - val_loss: 0.4780 - val_accuracy: 0.8000 - lr: 0.0010
Epoch 3/400
846/846 [==============================] - 17s 20ms/step - loss: 0.5259 - accuracy: 0.7338 - val_loss: 0.4477 - val_accuracy: 0.8160 - lr: 0.0010
Epoch 4/400
846/846 [==============================] - 17s 20ms/step - loss: 0.5053 - accuracy: 0.7502 - val_loss: 0.4617 - val_accuracy: 0.8060 - lr: 0.0010
Epoch 5/400
846/846 [==============================] - 17s 21ms/step - loss: 0.4885 - accuracy: 0.7577 - val_loss: 0.4721 - val_accuracy: 0.8040 - lr: 0.0010
Epoch 6/400
846/846 [==============================] - 18s 21ms/step - loss: 0.4714 - accuracy: 0.7685 - val_loss: 0.4529 - val_accuracy: 0.8080 - lr: 0.0010
Epoch 7/400
846/846 [==============================]

In [ ]:
# Evaluate the model on the test set
test_accuracy = model_1.evaluate(tfk.applications.xception.preprocess_input(X_test),y_test,verbose=0)[-1]
print('Test set accuracy %.4f' % test_accuracy)

Test set accuracy 0.8220


In [ ]:
model_1.save('Xception_NO_INVERSION_nobatchnorm')
del model_1

In [ ]:
ft_model = tfk.models.load_model('Xception_NO_INVERSION')

In [ ]:
ft_model.get_layer('xception').trainable = True
ft_model.get_layer('xception').summary()

Model: "xception"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 96, 96, 3)]          0         []                            
                                                                                                  
 block1_conv1 (Conv2D)       (None, 47, 47, 32)           864       ['input_1[0][0]']             
                                                                                                  
 block1_conv1_bn (BatchNorm  (None, 47, 47, 32)           128       ['block1_conv1[0][0]']        
 alization)                                                                                       
                                                                                                  
 block1_conv1_act (Activati  (None, 47, 47, 32)           0         ['block1_conv1_bn[0][0]

In [ ]:
# Freeze up to last N layers
N = 7    # last block
for i, layer in enumerate(ft_model.get_layer('xception').layers[:-N]):
  layer.trainable=False

In [ ]:
for i, layer in enumerate(ft_model.get_layer('xception').layers):
  print(layer.name, layer.trainable)

input_1 False
block1_conv1 False
block1_conv1_bn False
block1_conv1_act False
block1_conv2 False
block1_conv2_bn False
block1_conv2_act False
block2_sepconv1 False
block2_sepconv1_bn False
block2_sepconv2_act False
block2_sepconv2 False
block2_sepconv2_bn False
conv2d False
block2_pool False
batch_normalization False
add False
block3_sepconv1_act False
block3_sepconv1 False
block3_sepconv1_bn False
block3_sepconv2_act False
block3_sepconv2 False
block3_sepconv2_bn False
conv2d_1 False
block3_pool False
batch_normalization_1 False
add_1 False
block4_sepconv1_act False
block4_sepconv1 False
block4_sepconv1_bn False
block4_sepconv2_act False
block4_sepconv2 False
block4_sepconv2_bn False
conv2d_2 False
block4_pool False
batch_normalization_2 False
add_2 False
block5_sepconv1_act False
block5_sepconv1 False
block5_sepconv1_bn False
block5_sepconv2_act False
block5_sepconv2 False
block5_sepconv2_bn False
block5_sepconv3_act False
block5_sepconv3 False
block5_sepconv3_bn False
add_3 False
bl

In [ ]:
ft_model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.AdamW(), metrics='accuracy')

In [ ]:
history = ft_model.fit(
    x = tfk.applications.xception.preprocess_input(X_train), # We need to apply the preprocessing thought for the MobileNetV2 network
    y = y_train,
    batch_size = 16,
    epochs = 400,
    validation_data = (tfk.applications.xception.preprocess_input(X_val), y_val), # We need to apply the preprocessing thought for the MobileNetV2 network
    callbacks = [
        tfk.callbacks.EarlyStopping(monitor='val_accuracy', mode='max', patience=30, restore_best_weights=True),
        tfk.callbacks.ReduceLROnPlateau(monitor = 'val_accuracy', factor = 0.9, patience = 8, min_lr = 2e-6)
    ]
).history

Epoch 1/400
846/846 [==============================] - 45s 31ms/step - loss: 0.5213 - accuracy: 0.7436 - val_loss: 0.4153 - val_accuracy: 0.8140 - lr: 0.0010
Epoch 2/400
846/846 [==============================] - 25s 29ms/step - loss: 0.4561 - accuracy: 0.7836 - val_loss: 0.3882 - val_accuracy: 0.8400 - lr: 0.0010
Epoch 3/400
846/846 [==============================] - 22s 26ms/step - loss: 0.4201 - accuracy: 0.8104 - val_loss: 0.4096 - val_accuracy: 0.8180 - lr: 0.0010
Epoch 4/400
846/846 [==============================] - 22s 26ms/step - loss: 0.3859 - accuracy: 0.8320 - val_loss: 0.5028 - val_accuracy: 0.7940 - lr: 0.0010
Epoch 5/400
846/846 [==============================] - 22s 26ms/step - loss: 0.3603 - accuracy: 0.8455 - val_loss: 0.3977 - val_accuracy: 0.8220 - lr: 0.0010
Epoch 6/400
846/846 [==============================] - 23s 27ms/step - loss: 0.3163 - accuracy: 0.8639 - val_loss: 0.4056 - val_accuracy: 0.8260 - lr: 0.0010
Epoch 7/400
846/846 [==============================]

In [ ]:
# Evaluate the model on the test set
test_accuracy = ft_model.evaluate(tfk.applications.xception.preprocess_input(X_test),y_test,verbose=0)[-1]
print('Test set accuracy %.4f' % test_accuracy)

Test set accuracy 0.8120


In [ ]:
ft_model.save('Xception_NO_INVERSION_finetuned')
del ft_model